## Análisis exploratorio de los datos
![EDA](images/eda.png)

In [ ]:
# Importo las librerias que voy a utilizar en este notebook
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn
import nltk

from unicodedata import normalize
from transformers import pipeline
from transformers import AutoModelForSeqLM, AutoTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud


In [ ]:
df = pd.read_csv('../elmundo/input/df_elmundo.csv', encoding='latin1',
                 index_col=[0])

#### Nota: 
*Al final vamos a tratar todo el texto por igual, duplicando 'otros datos', tanto en titular como en noticia*

In [ ]:
# Rehacemos el dataset, y lo sobreescribimos, ya que lo tenemos en el csv.
df1 = df.iloc[:,1:]
df2 = df.loc[:,['Titulares','Otros_datos']]
df2.columns = ['Texto','Otros_datos']
df = pd.concat([df1,df2], axis=0).reset_index(drop=True)

In [ ]:
# Tenemos la columna de Texto, que es la que necesitamos, y otros datos que hemos scrappeado en las noticias.
musica_df.info()

In [ ]:
# Para poder guardar mas comodamente el csv y que no de problemas posteriormente al tokenizar, es pasar a minúscula,
# y quitar las tildes.

musica_df['Texto'] = musica_df.Texto.apply(lambda x: str.lower(x))

a,b = 'áéíóúü','aeiouu'
trans = str.maketrans(a,b)
musica_df['Texto'] = musica_df.Texto.apply(lambda x: x.translate(trans))


In [ ]:
# Se puede observar que en la columna Otros_datos hay diferentes datos separados por ('\n').
musica_df['Otros_datos'] = musica_df.Otros_datos.apply(lambda x: x.split('\n'))

In [ ]:
# Nos devuelve una lista con varios elementos.
musica_df['Clase'] = musica_df.Otros_datos.apply(lambda x: x[0])
musica_df['Coincidencia'] = musica_df.Otros_datos.apply(lambda x: x[2])
musica_df['varios'] = musica_df.Otros_datos.apply(lambda x: x[3])

In [ ]:
# Vemos que en el ultimo elemento, podemos sacar la fecha:
# La forma mas fácil que se me ha ocurrido ha sido con una expresión regular.

fechas = []
for i in musica_df['varios']:
    fechas.append(''.join(re.findall(r'[0-9/]', i)))

In [ ]:
musica_df['Fecha'] = fechas
musica_df['Fecha'].value_counts()

In [ ]:
# Para arreglar las fechas, bastaría con quedarnos con los primeros 10 caracteres de la cadena
fechas_bueno = []
for i in musica_df.Fecha:
    fechas_bueno.append(i[0:10])

In [ ]:
musica_df['Fecha'] = fechas_bueno
musica_df['Fecha'].value_counts()

In [ ]:
# Lo pasamos a formato datetime

musica_df['Fecha'] = pd.to_datetime(musica_df.Fecha)
musica_df.dtypes

In [ ]:
# Creamos una columna con los autores, que también estaban en el tercer elemento de la lista, luego aplico la misma
# solución que antes.
autores = []
for i in musica_df['varios']:
    autores.append(''.join(re.findall(r'[\sA-Z|]', i)))

In [ ]:
musica_df['Autores'] = autores

In [ ]:
# Limpiamos la columna coincidencia
coincidencias_num = []
for i in musica_df['Coincidencia']:
    coincidencias_num.append((''.join(re.findall(r'[0-9]', i))))

musica_df['Coincidencia_elmundo'] = coincidencias_num
musica_df['Coincidencia_elmundo'] = musica_df['Coincidencia_elmundo'].apply(lambda x: int(x))

**La columna coincidencia va de 0 a 100, asi que directamente he decido sacar la coincidencia relativa sobre el máximo**

In [ ]:
musica_df['Coincidencia_elmundo_relativa'] = musica_df['Coincidencia_elmundo']/musica_df['Coincidencia_elmundo'].max()

In [ ]:
# Y ya nos deshacemos las columnas que habiamos scrapeado.
musica_df_clean = musica_df.drop(columns=['Otros_datos','varios', 'Coincidencia'])
musica_df_clean

# BERT

*Según Wikipedia, cuna de la sabiduría popular:*

    - Bidirectional Encoder Representations from Transformers
    - BERT fue creado y publicado en 2018 por Jacob Devlin y sus colegas de Google.
    + Es una técnica basada en redes neuronales para el pre-entrenamiento del procesamiento del lenguaje natural (PLN) desarrollada por Google.

![](images/bert.jpg)

Lo que más me ha llamado la atención, que de igual manera, así lo expresa Wikipedia;
Es la capacidad de interpretar el sentido de la oración a través de los algoritmos de redes neuronales, y no se base en la interpretación básica de cada elemento en función de x características.

**Por este motivo, he visto fundamental utilizar transformers basados en BERT, en particular, el modelo de traducción, que es el modelo Helsinki-NLP para la traducción de inglés a español**


He comprobado que existen mas y mejores funcionalidades para NLP en inglés que en español, por lo que vamos a traducir el texto y trabajar con él, en inglés.

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-es-en')
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-es-en')

In [ ]:
translation = pipeline('translation_es_to_en', model=model, tokenizer=tokenizer)

In [ ]:
#Realizamos una prueba
text = 'Mi madre me hace la comida los domingos y yo no voy mucho a verla'
translated_text = translation(text, max_length=40)[0]['translation_text']
print(translated_text)

In [ ]:
# Vamos a crear una lista con los 1000 textos traducidos para posteriormente incorporarlos a dataframe
# Nota: lo he hecho un for y un contador, ya que tarda bastante tiempo. En mi este caso 27min.
%%time
english_texts = []


for j,i in enumerate(musica_df_clean['Texto']):
    english_texts.append(translation(i, max_length=100)[0]['translation_text'])
    if j % 10 == 0:
        print(j)

In [ ]:
musica_df_clean['English_text'] = english_texts

In [ ]:
#reordeno las columnas
cols = ['Fecha','Texto','English_text','Autores','Coincidencia_elmundo','Coincidencia_elmundo_relativa','Clase']
musica_df_clean = musica_df_clean[cols]

In [ ]:
#Solamente hay dos clases Noticia == 1 o Foto == 0

musica_df_clean['Label_clase'] = np.where(musica_df_clean['Clase'] == 'noticia',1,0)

# Estas clases estan extremadamente desbalanceadas, y tampoco parace que vaya a aportarnos mucho.

In [ ]:
# Veamos el score que le da huggingface con las tematicas mas buscadas en 2020 en españa y la nuestra.(inglés)
# Buscador de Google.
candidate_labels = ['music','coronavirus','elections']
classifier_zero = pipeline('zero-shot-classification')
text_prueba = 'i have been so sick this year'
classifier_zero(text_prueba,
               candidate_labels=candidate_labels)

In [ ]:
#De igual manera pongo un contador para ver como va avanzando
scores = []
labels = []
for j,i in enumerate(df['English_text']):
    dictionary = classifier_zero(i, candidate_labels=candidate_labels)
    scores.append(dictionary['scores'][0])
    labels.append(dictionary['labels'][0])
    if j % 20 == 0:
        print('Voy por el texto numero {}'.format(j))

In [ ]:
#Por ultimo el analisis de sentimiento, guardamos tambien las etiquetas para posteriormente multiplicar por -1
#los sentimientos negativos
classifier_sentiment = pipeline("sentiment-analysis")
scores_sentiment = []
labels_sentiment = []
for j,i in enumerate(df['English_text']):
    dictionary = classifier_sentiment(i)
    scores_sentiment.append(dictionary[0]['score'])
    labels_sentiment.append(dictionary[0]['label'])
    if j % 20 == 0:
        print('Voy por el texto numero {}'.format(j))

In [ ]:
# Metemos todas las listas al dataframe, el label de sentimiento lo transformamos a 1 si es positivo y -1 si es negativo
# Posteriormente multiplicamos este label de sentimiento por el score en el analisis, y de esta manera lo centramos en 0

musica_df_clean['Clasif_label_zero'] = labels
musica_df_clean['Scores_zero'] = scores
musica_df_clean['Analisis_sentimiento'] = scores_sentiment
musica_df_clean['Label_sentimiento'] = labels_sentiment

musica_df_clean['Label_sentimiento'] = np.where(df['Label_sentimiento'] == 'POSITIVE',1,-1)
musica_df_clean['Analisis_sentimiento_scd'] = df['Analisis_sentimiento'] * df['Label_sentimiento']

In [ ]:
# Lo guardamos en csv, para no tener que volver a repetir todo el entrenamiento.
musica_df_clean.to_csv('../elmundo/input/df_elmundo_fin.csv')

In [ ]:
# Volvemos a seleccionar las features.
df_english = pd.read_csv('../elmundo/input/musica_df_clean.csv',index_col=[0])
columns = ['Fecha','English_text','Autores','Analisis_sentimiento_scd','Scores_zero','Clasif_label_zero']

df_english = df[columns]
df_english

In [ ]:
# Vamos a ver cuantos registros tiene cada tematica
# Vemos cuantos registros tenemos de cada tematica.

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_palette('cubehelix')
sns.set_style('whitegrid')

fig, ax = plt.subplots(figsize=(10,7))
sns.histplot(x='Clasif_label_zero',data=df_english,ax=ax);

In [ ]:
top_sentimiento_clases = df_english.groupby('Clasif_label_zero')[['Analisis_sentimiento_scd']].mean().reset_index()
top_sentimiento_clases

In [ ]:
# Sentimiento positivo.

fig, ax = plt.subplots(figsize=(10,7))
sns.set_palette('inferno')
sns.set_style('whitegrid')
sns.barplot(x='Clasif_label_zero',y='Analisis_sentimiento_scd',data=top_sentimiento_clases, ax=ax)

In [ ]:
# DISTRIBUCION SENTIMIENTO
fig, ax = plt.subplots(figsize=(10,7))
sns.histplot(x='Analisis_sentimiento_scd', kde=True, data=df_english, ax=ax)

***Esta variable que hemos creado con Huggingface, identifica los sentimientos de las palabras de forma positiva o negativa dandoles un score a ambas clases ('Positivo y Negativo') sobre 1, el score que gane es el que se le adjudica a la palabra***

In [ ]:
# A través de esta función tokenizamos solo las palabras,

tokenizer = RegexpTokenizer(r'\w+')

df_english.loc[:,'English_text_token'] = df_english['English_text'].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
# Vamos a ver si existe alguna relación entre las tematicas y el analisis de sentimiento, cogiendo de muestra:
# los 20 autores con mas score de sentimiento positivo y los 20 autores con mas score de sentimiento negativo.

In [ ]:
autores_sentimiento = df_english.groupby('Autores')[['Analisis_sentimiento_scd']].mean().reset_index()

top_autores_positivos = autores_sentimiento.nlargest(20,'Analisis_sentimiento_scd')

top_autores_negativos = autores_sentimiento.nsmallest(20, 'Analisis_sentimiento_scd')

autores_top = pd.concat([top_autores_positivos,top_autores_negativos])
tema_pred = []
for i in autores_top['Autores']:
    tema_pred.append(df_english.loc[df_english['Autores']==i,'Clasif_label_zero'].mode()[0])

In [ ]:
autores_top['tema'] = tema_pred
sns.set_palette('tab20')
fig, ax = plt.subplots(figsize=(20,7))
sns.scatterplot(x='Autores',y='Analisis_sentimiento_scd',hue='tema',data=autores_top, ax=ax)
plt.xticks(rotation=45);

# Se pueden observar que a pesar de que las categorias estan balanceadas la tematica "coronavirus", abarca las 
# las tematicas principales de los autores valorados con mas sentimiento positivo y también las que mas sentimiento
# negativo se le atribuye.

##### Comentario
En este grafico no hemos podido observar gran relación basandonos en los comentarios top valoracion sentimiento y la tematica a la que pertenecen

In [ ]:
# Veamos si existe alguna relación del analisis de sentimiento dependiendo de la fecha
df_date_sentimiento = df[['Fecha','Analisis_sentimiento_scd']].set_index('Fecha')
df_date_sentimiento.index = pd.to_datetime(df_date_sentimiento.index)
monthly = df_date_sentimiento.resample('M').sum()
monthly.plot(style=[':'])
plt.title('Analisis sentimiento en los articulos por meses');

In [ ]:
weekly = df_date_sentimiento.resample('W').sum()
weekly.plot(style=[':'])
plt.title('Analisis sentimiento en los articulos por meses, semanalmente')

##### Comentario

Aquí podemos ver que la mayoria de artículos con sentimiento positivo se concentraron en los meses de verano con ciertas caídas puntuales.
Por lo general se pueden apreciar unas medias de sentimiento muy bajas durante el resto del año.

In [ ]:
# No puede ser menor de 0.33
df_labels = df_english.groupby('Clasif_label_zero')[['Scores_zero']].mean().reset_index()
sns.distplot(df_labels['Scores_zero'])

In [ ]:
sns.barplot(x='Clasif_label_zero',y='Scores_zero',data=df_labels)

Como era de esperar las clasificadas como musica tienen un mayor score, por lo que tienen un menor error.
La busqueda de estos artículos viene dada por la misma palabra 'musica'

In [ ]:
# Vamos a crear un array de las palabras sin las stopwords. La llamo Clean_words

def stop(raw):
    return [word.lower() for word in raw if word.lower() not in stopwords.words('english')]

df_english['Clean_words'] = df_english['English_text_token'].apply(lambda x: stop(x))

In [ ]:
# Aun que NLTK tiene una funcion propia, es facil hacerlo manualmente.
# Creamos un diccionario con la frecuencia de cada palabra.
dictionary = dict()

for i in df_english['Clean_words']:
    for j in i:
        if j in dictionary:
            dictionary[j] +=1
        else:
            dictionary[j] = 1

In [ ]:

palabras = []
veces = []
for (key,value) in dictionary.items():
    if value > 15:
        palabras.append(key)
        veces.append(value)
        
top_freq = pd.DataFrame({'palabras': palabras,
                        'veces':veces})      
top15_freq = top_freq.nlargest(15, 'veces')

sns.swarmplot(x='palabras', y='veces', data=top15_freq)

In [ ]:
print('Despues de quitar las stopwords y los signos de puntuación hay {} palabras diferentes'.format(len(dictionary))) 

In [ ]:
# Creamos una función para pintar un wordcloud con las palabras que tienen mas freq.
def cloud(text):
    wordcloud = WordCloud(background_color='black').generate(' '.join([i for i in text.str.upper()]))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title('Article Words')

cloud(df_english['English_text'])

## Stemming Words
Intentamos buscar los lexemas, raices comunes de las palabras

In [ ]:
# Para esto vamos a utilizar PorterStemmer de nltk.
stemmer = PorterStemmer()

def pstem(reg):
    return [stemmer.stem(word) for word in reg]
    

df_english['stems'] = df_english['Clean_words'].apply(lambda x: pstem(x))

In [ ]:
dictionary_stems = dict()

for i in df_english['stems']:
    for j in i:
        if j in dictionary_stems:
            dictionary_stems[j] +=1
        else:
            dictionary_stems[j] = 1

In [ ]:
print('Despues de quitar las stopwords y los signos de puntuación hay {} palabras diferentes'.format(len(dictionary_stems)))

In [ ]:
print('Podemos trabajar con {} palabras menos, gracias a los lexemas'.format(len(dictionary)-len(dictionary_stems)))

In [ ]:
df_english['stems'][0]

In [ ]:
# Ahora podríamos tagger también el tipo de palabra con la libreria nltk.pos_tag
# Vemos un ejemplo con la primera frase
tagged = nltk.pos_tag(df_english['stems'][0])
tagged

In [ ]:
# Al final creamos nuestro target, que consistirá en un clasificador binario partiendo de los datos que hemos conseguido,
# con el modelo bert diferenciando los textos que tengan relación con la palabra música y los que no.
# De este modo creamos la feature label con 1 si es musica o 0 si no lo es.

df_english['Label'] = np.where(df_english.Clasif_label_zero == 'music',1,0)

In [ ]:
df_model = df_english[['stems','Label']]

In [ ]:
df_model.to_csv('../elmundo/input/df_model.csv')